## Power-law analysis in gamma frequency band
Perform calculations of the Power Spectral Density (PSD) on log-log scale, where we evaluate the slope after have applied a low-pass filtering. We try several permutations to see if the evaluated slope of the PSD changes as a function of various parameters. As parameters we selects number of FFT lines: 256, 1024, 2048; order of the filter: 5 and 7 and fitting range: 34-80 Hz, and 50 - 80 Hz with 33 Hz and 50 hz cut-off filter values. We perform this analysis in the three cortical areas analysed only in wakefulness.

In [ ]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import os

In [ ]:
import warnings
warnings.filterwarnings("ignore")
warnings.resetwarnings()

In [ ]:
!pip install --upgrade pandas

In [ ]:
files = []

# Loop through files in the directory
for filename in os.listdir():
    if filename.endswith('W.csv'):
        
        files.append(filename)

In [ ]:
files

Perform high-pass filtering and calculate PSD on all the signals

In [ ]:
def norm(data):
  val = data/(data.sum())
  return val

def butter_highpass(cutoff, order):
    nyq = 0.5 * 200
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, order):
    
    b, a = butter_highpass(cutoff, order=order)
    y = signal.filtfilt(b, a, data)
    return y

def high_filtered(ar, cutoff, nperseg, order):

  def split_by_zero(arr):
    equals=np.isclose(arr, np.abs(arr).min())
    l=[]
    to_split=False
    for i in range(len(arr)):
        if equals[i] and i < len(arr)-1 and equals[i+1]:
            to_split=True
        elif not equals[i] and to_split:
            yield l
            l=[]
            to_split=False

        if not to_split:
            l.append(arr[i])
    yield l

  vector=[]
  s=[]
  for l in split_by_zero(ar):
    l=np.array(l)
    vector.append(l)
  f=[]
  for i in vector:
    x_filt = butter_highpass_filter(i, cutoff, order)
    fft=signal.welch(x_filt, fs=200, window="hann", nperseg=nperseg, nfft=nperseg, noverlap=0)

    f.append(norm(fft[1]))
    ftm=np.mean(f,axis=0)
    fstd=np.std(f,axis=0)

  return ftm

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
# Define a line for fitting the log-log PSD in 0.1-4 Hz frequency range
def line(a,x,b):
  return a*x+b

In [ ]:
cutoff = [33, 50]
orders = [5, 7]
fftlines = [256, 1024, 2048]
start_end = [[(44, 103), (176, 411), (350, 821)], [(65, 103), (258, 411), (514, 821)]]

params = []
fit = []

for file in files:
    
    df = pd.read_csv(file)
    
    # Iterate through fftlines, cutoff, order of the filter and fitting range

    for cut in cutoff:
                
        if cut == 33 or cut == 50: # for cutoff values equal to 33 Hz and 50 Hz let's set the fitting range 50 - 80 Hz
        
            for fl, se in zip(fftlines, start_end[1]):
            
                freq = np.fft.rfftfreq(fl, 1/200)
            
                for order in orders:
                    params.append(np.array([cut, order, fl, freq[se[0]],freq[se[1]-1], se[1]-se[0]]))
                    FFT_df1 = df.apply(high_filtered, args=[cut, fl, order])
                    k = curve_fit(line, np.log(freq[se[0]:se[1]]), 
                              np.log(np.mean(FFT_df1[se[0]:se[1]], axis=1)),
                              sigma=np.std(FFT_df1[se[0]:se[1]], axis=1) / np.mean(FFT_df1[se[0]:se[1]], axis=1),
                              absolute_sigma=True)
        
                    fit.append(k)
            
            
            if cut == 33: # for cutoff value equal to 33 Hz, the fitting range we set 34 - 80 Hz
            
                for fl, se in zip(fftlines, start_end[0]):

                    freq = np.fft.rfftfreq(fl, 1/200)
                
                    for order in orders:
                        params.append(np.array([cut, order, fl, freq[se[0]],freq[se[1]-1], se[1]-se[0]]))
                        FFT_df1 = df.apply(high_filtered, args=[cut, fl, order])
                        k = curve_fit(line, np.log(freq[se[0]:se[1]]), 
                                    np.log(np.mean(FFT_df1[se[0]:se[1]], axis=1)),
                                    sigma=np.std(FFT_df1[se[0]:se[1]], axis=1) / np.mean(FFT_df1[se[0]:se[1]], axis=1),
                                    absolute_sigma=True)
        
                        fit.append(k)
            
Post = fit[:18]
Prec = fit[18:36]
STG = fit[36:54]

In [ ]:
data = []
columns = ['Cut-off',"order", "FFT lines","Start","End","Num lines", "Beta", "Error", "Intercept", "Error1"]

# Loop through the values and append to the list
for (c, o, f, s, e, n), j in zip(params, fit):
        
    row_data = [c,o,f,s,e,n, j[0][0], np.sqrt(np.diag(j[1]))[0], j[0][1], np.sqrt(np.diag(j[1]))[1]]
    data.append(row_data)

# Create a DataFrame from the list
df = pd.DataFrame(data, columns=columns)

# Save obtained results as csv file
df.to_csv("Gamma_band_analysis.csv")

Statistical analysis of the parameters variability in three cortical areas.
Paired t_test performed by beta value comparisons with the mean beta value in a single cortical area.

In [ ]:
from scipy.stats import ttest_ind_from_stats as ttest

In [ ]:
stg_val = df.loc[36:54, ["Beta", "Error"]].reset_index(drop=True)
post_val = df.loc[:17, ["Beta", "Error"]].reset_index(drop=True)
prec_val = df.loc[18:35, ["Beta", "Error"]].reset_index(drop=True)

In [ ]:
def statistics(data):
    p_values = []
    t_tests = []

    for i,j in zip(data["Beta"], data["Error"]):
    
        t_test, p_value = ttest(data["Beta"].mean(), data["Beta"].std(), 18,
                                i, j, 18)
    
        t_tests.append(t_test)
        p_values.append(p_value)
        
    return t_tests, p_values

In [ ]:
colors = ["grey"]*3 + ["brown"]*3 + ["black"]*3
x_axis = [0, 1, 2]*3
f = [stg_val, post_val, prec_val]
fft_labels = [256, 1024, 2048]

fig, axes = plt.subplots(3, 1, figsize=(6, 10))

for ax, data in zip(axes, f):
    for index, color, pos in zip(range(0, len(data), 2), colors, x_axis):
        ax.errorbar(pos, np.abs(data.iloc[index]["Beta"]),
                    yerr=data.iloc[index]["Error"],
                    color=color, capsize=3, fmt="x", ms=10, ls="--", label="Legend")  # Replace "Legend" with your actual legend label

    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(fft_labels, size=14)
    ax.set_yticks([0.5, 1,1.5, 2, 2.5, 3, 3.5])
    ax.set_yticklabels([0.5,1.0,1.5, 2.0, 2.5, 3.0, 3.5], size=14)
    ax.set_ylabel("β", size=14)
    ax.set_ylim(0.5, 3.5)
    ax.grid()
plt.xlabel("Number of FFT lines", size=14)

plt.savefig("High_freq_anal.jpg", dpi=200, bbox_inches="tight")

In [ ]:
for index in zip(range(0, len(Post), 2)):
    print("Cut-off:{:.2f} Fit range: {:.2f}-{:.2f} (Hz)".format(p[index[0]][0],params[index[0]][3],params[index[0]][4]))